In [9]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime

In [69]:
#Get ETL_DATE for Incremental Ingestion (enabled if ETL_DATE set to 'CURRENT_DATE')
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

ETL_DATE = os.getenv('ETL_DATE')

In [71]:
# For Incremental Ingestion (enabled if ETL_DATE set to 'CURRENT_DATE')
if ETL_DATE == 'CURRENT_DATE':
    ETL_DATE = datetime.today().strftime('%Y%m%d')
else:
   ETL_DATE = '20250322'
    
print(ETL_DATE)

20250328


In [44]:
data = pd.read_csv(f"data/silver_{ETL_DATE}_Airline_Reviews_Preprocessed.csv")
data.shape

(52, 22)

In [48]:
print(len(data))
data.head()

52


,RowId,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,Id
0,0,Aer Lingus,2.0,“terrible customer service”,2025-03-26,False,Absolutely terrible customer service. Tried t...,NaN,NaN,Couple Leisure,...,2025-03-01,2.0,3.0,3.0,1.0,NaN,3.0,1,no,661172569262a3a1c15d87e9459a0fefc8664914266518...
1,1,Air France,10.0,"""excellent customer service""",2025-03-28,True,My son was travelling on Kids Solo as an un...,NaN,NaN,Family Leisure,...,2025-03-01,NaN,NaN,NaN,5.0,NaN,NaN,5,yes,925fcfda402034e92b439424f9eb35bec83154e497e641...
2,2,Air India,3.0,“it was not a pleasant experience”,2025-03-26,False,Recently travelled on Air India flight from T...,NaN,NaN,Family Leisure,...,2025-03-01,1.0,3.0,1.0,4.0,1.0,1.0,1,no,c09a8175ab7eda10f0fddbbdc6227c44836d964901af2c...
3,3,Air India Express,1.0,"""not accountable for any damage""",2025-03-23,True,No proper communication and very casual app...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,2025-03-01,1.0,1.0,1.0,1.0,1.0,1.0,1,no,455e41f45426cc1ebd206caba0945f054613394cb208c5...
4,4,AirAsia X,2.0,"""flight is delay for 20 hours""",2025-03-24,True,"Worse experience, my flight suppose to depa...",NaN,NaN,Business,...,2025-03-01,1.0,1.0,1.0,1.0,NaN,NaN,1,no,a13ee7a23ddc22c11deadf130dbe3b23384427e2fd5e01...


### Hugging Face - cardiffnlp/twitter-roberta-base-sentiment-latest

In [51]:
from transformers import pipeline
RBSL_PN_MODEL = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
RBSL_PN_MODEL('I am happy')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'label': 'positive', 'score': 0.9614965319633484}]

In [52]:

# Make sure reviews are in the right format (list of strings)
review_texts = data['Review'].fillna("").astype(str).tolist()

# Process in batches
batch_size = 32
all_results = []

for i in tqdm(range(0, len(review_texts), batch_size), desc="Processing reviews"):
    batch = review_texts[i:i + batch_size]
    try:
        batch_results = RBSL_PN_MODEL(
            batch,
            truncation=True,
            max_length=512
        )
        all_results.extend(batch_results)
    except Exception as e:
        print(f"Error in batch {i}-{i+batch_size}: {e}")
        all_results.extend([{'label': 'ERROR', 'score': 0.0} for _ in batch])

# Convert to DataFrame
sentiments_df = pd.DataFrame(all_results)
sentiments_df.columns = ['sentiment_label', 'sentiment_scores']

Processing reviews:   0%|          | 0/2 [00:00<?, ?it/s]

In [53]:
print(len(sentiments_df))
sentiments_df.head()

52


,sentiment_label,sentiment_scores
0,negative,0.948760
1,neutral,0.701676
2,negative,0.710657
3,negative,0.902924
4,negative,0.841998


#### Merge back review text labels to original dataframe

In [56]:
df_merged = data.join(sentiments_df, how='left')
print(len(df_merged))
df_merged.head()

52


,RowId,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,Id,sentiment_label,sentiment_scores
0,0,Aer Lingus,2.0,“terrible customer service”,2025-03-26,False,Absolutely terrible customer service. Tried t...,NaN,NaN,Couple Leisure,...,3.0,3.0,1.0,NaN,3.0,1,no,661172569262a3a1c15d87e9459a0fefc8664914266518...,negative,0.948760
1,1,Air France,10.0,"""excellent customer service""",2025-03-28,True,My son was travelling on Kids Solo as an un...,NaN,NaN,Family Leisure,...,NaN,NaN,5.0,NaN,NaN,5,yes,925fcfda402034e92b439424f9eb35bec83154e497e641...,neutral,0.701676
2,2,Air India,3.0,“it was not a pleasant experience”,2025-03-26,False,Recently travelled on Air India flight from T...,NaN,NaN,Family Leisure,...,3.0,1.0,4.0,1.0,1.0,1,no,c09a8175ab7eda10f0fddbbdc6227c44836d964901af2c...,negative,0.710657
3,3,Air India Express,1.0,"""not accountable for any damage""",2025-03-23,True,No proper communication and very casual app...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,1.0,1.0,1.0,1.0,1.0,1,no,455e41f45426cc1ebd206caba0945f054613394cb208c5...,negative,0.902924
4,4,AirAsia X,2.0,"""flight is delay for 20 hours""",2025-03-24,True,"Worse experience, my flight suppose to depa...",NaN,NaN,Business,...,1.0,1.0,1.0,NaN,NaN,1,no,a13ee7a23ddc22c11deadf130dbe3b23384427e2fd5e01...,negative,0.841998


#### Persist to CSV file

In [58]:
df_merged.to_csv(f"data/silver_{ETL_DATE}_Airline_Reviews_Sentiment.csv", index=False, date_format='%Y-%m-%d')